# Biclustering documents with the Spectral Co-clustering algorithm

In [2]:
import operator
from collections import defaultdict
from time import time
from typing import Callable

import numpy as np

from sklearn.cluster import MiniBatchKMeans, SpectralCoclustering
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.cluster import v_measure_score

In [3]:
def number_normalizer(tokens):
    """
    모든 숫자 토큰을 placehoder에 매핑한다.
    많은 application에서 숫자로 시작하는 토근은 직접적으로 유용하지는 않지만 토큰이 존재한다는 사실이 관련될 수는 있다.
    이러한 형태의 차원 감소를 적용함으로 일부 방법은 더 나은 성능을 발휘할 수 있다.
    """
    return ("#NUMBER" if token[0].isdigit() else token for token in tokens)

In [4]:
class NumberNormalizingVectorizer(TfidfVectorizer):
    def build_tokenizer(self):
        tokenize = super().build_tokenizer()
        return lambda doc: list(number_normalizer(tokenize(doc)))
    
# comp.os.ms-windows.misc 제외
categories = [
    "alt.atheism",
    "comp.graphics",
    "comp.sys.ibm.pc.hardware",
    "comp.sys.mac.hardware",
    "comp.windows.x",
    "misc.forsale",
    "rec.autos",
    "rec.motorcycles",
    "rec.sport.baseball",
    "rec.sport.hockey",
    "sci.crypt",
    "sci.electronics",
    "sci.med",
    "sci.space",
    "soc.religion.christian",
    "talk.politics.mideast",
    "talk.politics.misc",
    "talk.religion.misc",
]
newsgroups = fetch_20newsgroups(categories=categories)
y_true = newsgroups.target

vectorizer = NumberNormalizingVectorizer(stop_words="english", min_df=5)
cocluster = SpectralCoclustering(
    n_clusters=len(categories), svd_method="arpack", random_state=0
)
kmeans = MiniBatchKMeans(
    n_clusters=len(categories), batch_size=20000, random_state=0, n_init=3
)

print("Vectorizing...")
X = vectorizer.fit_transform(newsgroups.data)

print("Coclustering...")
start_time = time()
cocluster.fit(X)
y_cocluster = cocluster.row_labels_
print(
    "Done in {:.2f}s. V-measure: {:.4f}".format(time() - start_time, v_measure_score(y_cocluster, y_true))
)

print("MiniBatchKmeans...")
start_time = time()
y_kemans = kmeans.fit_predict(X)
print(
     "Done in {:.2f}s. V-measure: {:.4f}".format(time() - start_time, v_measure_score(y_kemans, y_true))
)

feature_names = vectorizer.get_feature_names_out()
document_names = list(newsgroups.target_names[i] for i in newsgroups.target)

Vectorizing...
Coclustering...
Done in 0.54s. V-measure: 0.4487
MiniBatchKmeans...
Done in 1.95s. V-measure: 0.2690


In [5]:
def bicluster_ncut(i):
    rows, cols = cocluster.get_indices(i)
    if not (np.any(rows) and np.any(cols)):
        import sys

        return sys.float_info.max
    row_complement = np.nonzero(np.logical_not(cocluster.rows_[i]))[0]
    col_complement = np.nonzero(np.logical_not(cocluster.columns_[i]))[0]
    # X[rows[:, np.newaxis], cols].sum()이 scipy <= 0.16  
    weight = X[rows][:, cols].sum()
    cut = X[row_complement][:, cols].sum() + X[rows][:, col_complement].sum()
    return cut / weight

In [7]:
def most_common(d):
    """
    가장 높은 값을 가진 defaultdict(int)의 항목이다.
    Python >= 2.7에서 Counter.most_common과 비슷하다.
    """    
    return sorted(d.items(), key=operator.itemgetter(1), reverse=True)

bicluster_ncuts = list(bicluster_ncut(i) for i in range(len(newsgroups.target_names)))
best_idx = np.argsort(bicluster_ncuts)[:5]

print()
print("Best biclusters:")
print("----------------")
for idx, cluster in enumerate(best_idx):
    n_rows, n_cols = cocluster.get_shape(cluster)
    cluster_docs, cluster_words = cocluster.get_indices(cluster)
    if not len(cluster_docs) or not len(cluster_words):
        continue

    # categories
    counter = defaultdict(int)
    for i in cluster_docs:
        counter[document_names[i]] += 1
    cat_string = ", ".join(
        "{:.0f}% {}".format(float(c) / n_rows * 100, name)
        for name, c in most_common(counter)[:3]
    )

    # words
    out_of_cluster_docs = cocluster.row_labels_ != cluster
    out_of_cluster_docs = np.where(out_of_cluster_docs)[0]
    word_col = X[:, cluster_words]
    word_scores = np.array(
        word_col[cluster_docs, :].sum(axis=0)
        - word_col[out_of_cluster_docs, :].sum(axis=0)
    )
    word_scores = word_scores.ravel()
    important_words = list(
        feature_names[cluster_words[i]] for i in word_scores.argsort()[:-11:-1]
    )

    print("bicluster {} : {} documents, {} words".format(idx, n_rows, n_cols))
    print("categories   : {}".format(cat_string))
    print("words        : {}\n".format(", ".join(important_words)))


Best biclusters:
----------------
bicluster 0 : 8 documents, 6 words
categories   : 100% talk.politics.mideast
words        : cosmo, angmar, alfalfa, alphalpha, proline, benson

bicluster 1 : 1092 documents, 2929 words
categories   : 30% talk.politics.mideast, 27% soc.religion.christian, 26% alt.atheism
words        : god, jesus, christians, atheists, morality, kent, sin, belief, objective, resurrection

bicluster 2 : 2225 documents, 2862 words
categories   : 18% comp.sys.mac.hardware, 16% comp.sys.ibm.pc.hardware, 16% sci.electronics
words        : voltage, shipping, receiver, circuit, compression, digital, processing, scope, baalke, package

bicluster 3 : 1505 documents, 3837 words
categories   : 24% talk.politics.misc, 18% sci.med, 17% soc.religion.christian
words        : geb, banks, gordon, drugs, kaldis, dyer, br, surrender, noring, n3jxp

bicluster 4 : 1722 documents, 2666 words
categories   : 27% rec.motorcycles, 23% rec.autos, 13% misc.forsale
words        : bike, car, dod, r